In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# Cargar los datos
data = pd.read_csv('cars.csv', sep=';')



In [14]:
# 1. LIMPIEZA DE DATOS

# Reemplazar valores "NO DATA" o similares con NaN
data.replace(["NO DATA", "N/A", "n/a", "?", "", " "], np.nan, inplace=True)

# Rellenar valores faltantes (media para numéricos, moda para categóricos)
for column in data.columns:
    if data[column].dtype == 'object':  # variables categóricas
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:  # variables numéricas
        data[column].fillna(data[column].mean(), inplace=True)

# 1.2 Eliminación de outliers usando el rango intercuartil (IQR)
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Aplicamos el tratamiento de outliers solo a las columnas numéricas relevantes
numeric_cols = ['EDAD_COCHE', 'COSTE_VENTA', 'km_anno', 'Edad Cliente', 'Tiempo']
for col in numeric_cols:
    data = remove_outliers(data, col)

# 2. CODIFICACIÓN DE VARIABLES CATEGÓRICAS
# Usamos LabelEncoder para convertir variables categóricas en formato numérico
categorical_cols = data.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Guardar el codificador para su uso futuro

# 3. NORMALIZACIÓN Y ESCALADO
# Escalado de las variables numéricas
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Guardar el archivo ya procesado en un nuevo archivo CSV limpio
data.to_csv('cars_limpio.csv', index=False)

print("Archivo limpio guardado como 'ruta_a_tu_archivo_limpio.csv'")



Archivo limpio guardado como 'ruta_a_tu_archivo_limpio.csv'


In [15]:
#3. Desarrollo del Modelo de Propensión:
from sklearn.model_selection import train_test_split
data=pd.read_csv('cars_limpio.csv')
X = data.drop('Mas_1_coche', axis=1)  # Todas las variables menos el objetivo
y = data['Mas_1_coche']               # Variable objetivo

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Modelo de regresión logística
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score


logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_log = logistic_model.predict(X_test)
print(f"Accuracy (Regresión Logística): {accuracy_score(y_test, y_pred_log)}")
print(f"AUC-ROC: {roc_auc_score(y_test, logistic_model.predict_proba(X_test)[:, 1])}")
print(classification_report(y_test, y_pred_log))


Accuracy (Regresión Logística): 0.9999400587424324
AUC-ROC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12125
           1       1.00      1.00      1.00      4558

    accuracy                           1.00     16683
   macro avg       1.00      1.00      1.00     16683
weighted avg       1.00      1.00      1.00     16683



In [19]:
# Modelo de Árbol de Decisión
from sklearn.tree import DecisionTreeClassifier


tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred_tree = tree_model.predict(X_test)
print(f"Accuracy (Árbol de Decisión): {accuracy_score(y_test, y_pred_tree)}")
print(f"AUC-ROC: {roc_auc_score(y_test, tree_model.predict_proba(X_test)[:, 1])}")
print(classification_report(y_test, y_pred_tree))


Accuracy (Árbol de Decisión): 1.0
AUC-ROC: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12125
           1       1.00      1.00      1.00      4558

    accuracy                           1.00     16683
   macro avg       1.00      1.00      1.00     16683
weighted avg       1.00      1.00      1.00     16683



In [20]:
correlations = X.corrwith(y)
print(correlations)
#Aqui vemos que tiempo tiene una correlacion extremadamente alta (0,9379) por lo q podria ser una fuga de informacion

CODE              -0.001280
PRODUCTO          -0.022678
TIPO_CARROCERIA   -0.032507
COMBUSTIBLE       -0.030213
Potencia_         -0.018317
TRANS             -0.001697
FORMA_PAGO        -0.007551
ESTADO_CIVIL      -0.029561
GENERO             0.014677
OcupaciOn          0.002798
PROVINCIA         -0.003128
Campanna1          0.114416
Campanna2          0.005062
Campanna3         -0.021214
Zona _Renta       -0.270953
REV_Garantia            NaN
Averia_grave      -0.017584
QUEJA_CAC         -0.036158
EDAD_COCHE         0.020459
COSTE_VENTA        0.179426
km_anno            0.134452
Revisiones         0.014963
Edad Cliente       0.014821
Tiempo             0.937908
dtype: float64


In [21]:
# Eliminar la columna 'Tiempo' debido a la alta correlación
X = X.drop('Tiempo', axis=1)

# Volver a dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Reentrenar el modelo (por ejemplo, regresión logística)
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Evaluar el modelo
y_pred_log = logistic_model.predict(X_test)
print(f"Accuracy (Regresión Logística): {accuracy_score(y_test, y_pred_log)}")
print(f"AUC-ROC: {roc_auc_score(y_test, logistic_model.predict_proba(X_test)[:, 1])}")
print(classification_report(y_test, y_pred_log))


Accuracy (Regresión Logística): 0.7415932386261463
AUC-ROC: 0.7348230142538553
              precision    recall  f1-score   support

           0       0.76      0.95      0.84     12125
           1       0.58      0.19      0.29      4558

    accuracy                           0.74     16683
   macro avg       0.67      0.57      0.57     16683
weighted avg       0.71      0.74      0.69     16683



In [22]:
#Parece que esta sobreajustado por lo que utilizaremos la validacion cruzada

from sklearn.model_selection import cross_val_score

# K-Fold Cross Validation con Regresión Logística
scores = cross_val_score(logistic_model, X, y, cv=5, scoring='accuracy')
print(f"Exactitud media (Validación Cruzada): {scores.mean()}")


Exactitud media (Validación Cruzada): 0.7360141854616142
